In [16]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [17]:
dataset_path = "/content/drive/MyDrive/RAVDESS/audio_speech_actors_01-24"


In [20]:
!unzip "/content/drive/MyDrive/audio_speech_actors_01-24.zip" -d "/content/"


Archive:  /content/drive/MyDrive/audio_speech_actors_01-24.zip
  inflating: /content/Actor_01/03-01-01-01-01-01-01.wav  
  inflating: /content/Actor_01/03-01-01-01-01-02-01.wav  
  inflating: /content/Actor_01/03-01-01-01-02-01-01.wav  
  inflating: /content/Actor_01/03-01-01-01-02-02-01.wav  
  inflating: /content/Actor_01/03-01-02-01-01-01-01.wav  
  inflating: /content/Actor_01/03-01-02-01-01-02-01.wav  
  inflating: /content/Actor_01/03-01-02-01-02-01-01.wav  
  inflating: /content/Actor_01/03-01-02-01-02-02-01.wav  
  inflating: /content/Actor_01/03-01-02-02-01-01-01.wav  
  inflating: /content/Actor_01/03-01-02-02-01-02-01.wav  
  inflating: /content/Actor_01/03-01-02-02-02-01-01.wav  
  inflating: /content/Actor_01/03-01-02-02-02-02-01.wav  
  inflating: /content/Actor_01/03-01-03-01-01-01-01.wav  
  inflating: /content/Actor_01/03-01-03-01-01-02-01.wav  
  inflating: /content/Actor_01/03-01-03-01-02-01-01.wav  
  inflating: /content/Actor_01/03-01-03-01-02-02-01.wav  
  inflati

In [21]:
dataset_path = "/content/audio_speech_actors_01-24"


In [22]:
import glob

files = glob.glob(dataset_path + "/**/*.wav", recursive=True)
print("Total files:", len(files))


Total files: 1440


In [23]:
!pip install librosa


In [24]:
import os
import numpy as np
import librosa
import pandas as pd


In [25]:
emotion_map = {
    "01": "neutral",
    "02": "calm",
    "03": "happy",
    "04": "sad",
    "05": "angry",
    "06": "fear",
    "07": "disgust",
    "08": "surprise"
}


In [26]:
def extract_features(file_path):
    audio, sample_rate = librosa.load(file_path, duration=3)
    mfcc = librosa.feature.mfcc(y=audio, sr=sample_rate, n_mfcc=40)
    return np.mean(mfcc.T, axis=0)


In [27]:
features = []
labels = []

dataset_path = "/content/audio_speech_actors_01-24"

for root, dirs, files in os.walk(dataset_path):
    for file in files:
        if file.endswith(".wav"):
            emotion_code = file.split("-")[2]
            emotion = emotion_map[emotion_code]

            file_path = os.path.join(root, file)
            data = extract_features(file_path)

            features.append(data)
            labels.append(emotion)


In [28]:
X = np.array(features)
y = np.array(labels)


In [29]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
y = le.fit_transform(y)


In [30]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)


In [31]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

model = Sequential()

model.add(Dense(256, activation='relu', input_shape=(X.shape[1],)))
model.add(Dense(128, activation='relu'))
model.add(Dense(len(np.unique(y)), activation='softmax'))

model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)


/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [32]:
model.fit(X_train, y_train, epochs=30, batch_size=32)


Epoch 1/30
36/36 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.1496 - loss: 20.0955
Epoch 2/30
36/36 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.1945 - loss: 3.6370
Epoch 3/30
36/36 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.2705 - loss: 2.4627
Epoch 4/30
36/36 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.2814 - loss: 2.5869
Epoch 5/30
36/36 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.3046 - loss: 2.0847
Epoch 6/30
36/36 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.3060 - loss: 2.1696
Epoch 7/30
36/36 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.3411 - loss: 1.9855
Epoch 8/30
36/36 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.3403 - loss: 2.4266
Epoch 9/30
36/36 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.2868 - loss: 3.0397
Epoch 10/30
36/36 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.2766 - loss: 2.9041
Epoch 11/30
36/36 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.3693 - loss: 1.9673
Epoch 12/30
36/36 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.3442 - l

In [33]:
loss, accuracy = model.evaluate(X_test, y_test)
print("Accuracy:", accuracy)


9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.3078 - loss: 2.7745  
Accuracy: 0.3576388955116272
